# ProPyCore SDK Examples: Documents

This notebook contains snippets from the `documents` submodules: `folders.py` and `files.py`.

In [6]:
import os
import dotenv
import json

import ProPyCore

## Setup
You will need to create the connection to your Procore app and then get the details for your company and project.

### Connection to Procore App
Ensure you have a `.env` file with the following information included:
* `CLIENT_ID`: your data connection app's client ID
* `CLIENT_SECRET`: your data connection app's client secret

In [7]:
dotenv.load_dotenv()
connection = ProPyCore.procore.Procore(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    redirect_uri="urn:ietf:wg:oauth:2.0:oob", # default for data connection apps
    oauth_url="https://app.procore.com", # default for data connection apps
    base_url="https://app.procore.com" # default for data connection apps
)

### Get Company Details
Use the cell below to specify your company name

In [8]:
company_name = "Rogers-O`Brien Construction"

Now get the company details

In [9]:
company = connection.companies.find(identifier=company_name)

### `projects.get()`
Get a list of all projects where the app has been installed. 

Now use the ProPyCore `connection` variable to get the company and project details.

In [11]:
company = connection.companies.find(identifier=company_name)
projects = connection.projects.get(
    company_id=company["id"]
)
for project in projects:
    print(f"Project: {project['name']} ({project['id']})")

Project: 1301 South Lamar (1668030)
Project: 24th & Rio Grande Student Housing (2289314)
Project: 315 College Main Apartments (1642207)
Project: 400 W.15th Elevator Mods (2023731)
Project: Mexican American Cultural Center, Phase 2 (1494731)
Project: Sandbox Test Project (2783683)
Project: South Laredo Multifamily (2430207)
Project: TSTC Waco CCAP (2602910)


---